In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import warnings
from tabulate import tabulate
import xgboost as xgb
import dask.dataframe as dd
warnings.filterwarnings("ignore")

In [2]:
iv_features = pd.read_csv("../reports/iv_features.csv")

In [3]:
# train.drop('WOE_target',axis=1, inplace=True)
features = iv_features["useful"].to_list()
features.remove("target")
len(features)

102

In [12]:
#First slice
features = features[:50]
features.insert(0, "customer_ID")

In [4]:
#Second slice
features = features[50:]
features.insert(0, "customer_ID")

In [5]:
train = pd.read_parquet("../data/processed/train_withlabels.parquet")

In [4]:
test = pd.read_parquet("../data/raw/test.parquet")

NameError: name 'pd' is not defined

In [7]:
test=test[features]

In [ ]:
# # Features for balancing
# features = train.columns.to_list()
# features = features[1:68]

## Preparing test set for submission

In [8]:

def iv_woe(data, feature, target,bins=10, show_woe=False, show_iv= False, balance= True):
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    #Extract Column Names
    cols = [feature]
    #Run WOE and IV on all the independent variables
    for ivars in cols:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop', precision=7)
            data[ivars]= binned_x
            data[ivars] = data[ivars].astype(str)
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
            d0 = d0.astype({"x": str})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
            data[ivars] = data[ivars]
    
    d = d0.groupby("x", as_index=False, dropna=False).agg({"y": ["count", "sum"]})
    d.columns = ['Cutoff', 'N', 'Events']
    
    d.insert(loc=0, column='Variable', value=ivars)

    d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
    d['Non-Events'] = d['N'] - d['Events']
    d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
    d['WoE'] = np.log(d['% of Non-Events']/d['% of Events'])
    d['IV'] = d['WoE'] * (d['% of Non-Events']-d['% of Events'])
    
    if (d["Cutoff"].dtypes == "object"):
            d.replace({"nan":" NaN,NaN "}, inplace=True)
            d["min"] = d["Cutoff"].apply(lambda x: x.split(',')[0][1:7])
            d["max"] = d["Cutoff"].apply(lambda x: x.split(',')[1][:-1])
            d.replace({"NaN":np.NaN}, inplace=True)
            d["min"] = d["min"].astype("float")
            d["max"] = d["max"].astype("float")
    else:
        d["min"] = d["Cutoff"].apply(lambda x: x)
        d['max'] = d["Cutoff"].apply(lambda x: x)
        
    temp = pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
    newDF=pd.concat([newDF,temp], axis=0)
    woeDF=pd.concat([woeDF,d], axis=0)
    if balance:
        balance_col = ["WoE","min", "max"]
        woeDF = woeDF[balance_col]
        return woeDF
    return newDF, woeDF

In [9]:
variaveis = test[features].columns
#df1 = {}
l = 0
for col in variaveis:
    if col == "target" or col == "customer_ID" or col == "S_2": continue
    else:
        print("WoE e IV: {}".format(col))
        df = iv_woe(train, col, "target",)
        # df1[l] = df
        # df1[l]["nome"] = "WOE_"+col
        print(l)
        l = l + 1
        #print(tabulate(df, headers="keys"))
        #print("IV score: {:.2f}".format(iv))
        print("\n")
        i = 0
        for i in range(0,len(df)):
            if df[i:i+1]["min"].isna().values:
                test.loc[test[col].isna(), col] = df.iloc[i,0]
            else:
                test.loc[test[col].between(df.iloc[i,1], df.iloc[i,2], inclusive=True), col] =  df.iloc[i,0]
        # del df
        gc.collect()



WoE e IV: R_20
0


WoE e IV: R_21
1


WoE e IV: D_89
2


WoE e IV: D_91
3


WoE e IV: D_92
4


WoE e IV: D_94
5


WoE e IV: R_24
6


WoE e IV: D_96
7


WoE e IV: S_23
8


WoE e IV: S_25
9


WoE e IV: S_26
10


WoE e IV: D_102
11


WoE e IV: D_103
12


WoE e IV: D_104
13


WoE e IV: D_105
14


WoE e IV: D_107
15


WoE e IV: R_26
16


WoE e IV: R_27
17


WoE e IV: D_112
18


WoE e IV: S_27
19


WoE e IV: D_113
20


WoE e IV: D_114
21


WoE e IV: D_115
22


WoE e IV: D_116
23


WoE e IV: D_117
24


WoE e IV: D_118
25


WoE e IV: D_119
26


WoE e IV: D_120
27


WoE e IV: D_121
28


WoE e IV: D_122
29


WoE e IV: D_124
30


WoE e IV: D_125
31


WoE e IV: D_126
32


WoE e IV: D_127
33


WoE e IV: D_128
34


WoE e IV: D_129
35


WoE e IV: B_41
36


WoE e IV: B_42
37


WoE e IV: D_130
38


WoE e IV: D_132
39


WoE e IV: D_133
40


WoE e IV: D_134
41


WoE e IV: D_135
42


WoE e IV: D_136
43


WoE e IV: D_137
44


WoE e IV: D_138
45


WoE e IV: D_139
46


WoE e IV: D_140
47


WoE e IV: D_141
48

In [15]:
t1.to_parquet("../data/final/test.parquet")

In [7]:
test= pd.read_parquet("../data/final/test.parquet")

In [8]:
a=  test.drop_duplicates()

In [9]:
len(a)

11363751

In [14]:
submission

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


In [16]:
for i in range(0, len(submission)):
    a = test.loc[test["customer_ID"] == submission.iloc[i,0]]

KeyboardInterrupt: 

In [ ]:
a.head()

In [ ]:
len(a)

In [17]:
test.set_index("customer_ID")

,D_39,D_41,D_43,B_5,R_2,D_46,B_8,D_50,R_3,P_3,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_145
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.202999,-1.429173,0.501762,-0.335899,0.132993,0.320558,-0.717255,-0.258674,0.379859,-0.878053,...,0.071138,0.071138,0.071138,0.071138,-0.517741,0.047089,-0.517741,0.134366,-0.517741,-0.448826
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.202999,-1.429173,0.501762,-0.146611,0.132993,0.320558,-0.733333,-0.258674,0.379859,-0.878053,...,0.071138,0.071138,0.071138,0.071138,0.151702,0.047089,0.057189,0.134366,0.151680,-0.448826
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.202999,-1.429173,0.501762,1.357446,0.132993,0.320558,-0.717255,-0.258674,0.379859,-0.878053,...,0.071138,0.071138,0.071138,0.071138,0.151702,0.047089,0.057189,0.134366,0.151680,-0.448826
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.202999,-1.429173,0.501762,1.357446,0.132993,0.320558,-0.733333,-0.258674,0.379859,-0.878053,...,0.071138,0.071138,0.071138,0.071138,0.151702,0.047089,0.057189,0.134366,0.151680,-0.448826
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,-0.093474,-1.429173,0.501762,1.357446,0.132993,-0.706420,-0.720294,-0.258674,0.379859,0.663292,...,0.071138,0.071138,0.071138,0.071138,0.151702,0.047089,0.057189,0.134366,0.151680,-0.448826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61cceb803ea8ec37634d,0.202999,-1.429173,0.501762,1.357446,0.132993,-0.145331,-0.718892,-0.258674,-0.762993,0.663292,...,0.071138,0.071138,0.071138,0.071138,0.151702,0.047089,0.057189,0.134366,0.151680,-0.448826
fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61cceb803ea8ec37634d,0.202999,-1.429173,0.501762,1.357446,0.132993,-0.145331,-0.718892,-0.258674,-0.762993,0.663292,...,0.071138,0.071138,0.071138,0.071138,0.151702,0.047089,0.057189,0.134366,0.151680,-0.448826
fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61cceb803ea8ec37634d,0.202999,-1.429173,0.501762,1.357446,0.132993,-0.012077,-0.720294,-0.258674,-0.762993,0.579708,...,0.071138,0.071138,0.071138,0.071138,0.151702,0.047089,0.057189,0.134366,0.151680,-0.448826


In [11]:
submission = pd.read_csv("../data/raw/sample_submission.csv")

In [8]:
customers = test[["customer_ID"]].drop_duplicates().sort_index().values.flatten()

_____________________________________________

In [15]:
hex_to_int = lambda x: int(x, 16)
test["customer_ID"].str[-16:].apply(hex_to_int).astype('int64')

0           8717704911770597815
1           8717704911770597815
2           8717704911770597815
3           8717704911770597815
4           8717704911770597815
                   ...         
11363757   -1477111782539041971
11363758   -1477111782539041971
11363759   -1477111782539041971
11363760   -1477111782539041971
11363761   -1477111782539041971
Name: customer_ID, Length: 11363762, dtype: int64

In [2]:
def read_file(path = '', usecols = None):
    # LOAD DATAFRAME
    if usecols is not None: df = pd.read_parquet(path, columns=usecols)
    else: df = pd.read_parquet(path)
    # REDUCE DTYPE FOR CUSTOMER AND DATE
    # SORT BY CUSTOMER AND DATE (so agg('last') works correctly)
    df = df.sort_values(['customer_ID'])
    df = df.reset_index(drop=True)
    
    # FILL NAN
    print('shape of data:', df.shape)
    
    return df


In [3]:
# CALCULATE SIZE OF EACH SEPARATE TEST PART
def get_rows(customers, test, NUM_PARTS = 4, verbose = ''):
    chunk = len(customers)//NUM_PARTS
    if verbose != '':
        print(f'We will process {verbose} data as {NUM_PARTS} separate parts.')
        print(f'There will be {chunk} customers in each part (except the last part).')
        print('Below are number of rows in each part:')
    rows = []

    for k in range(NUM_PARTS):
        if k==NUM_PARTS-1: cc = customers[k*chunk:]
        else: cc = customers[k*chunk:(k+1)*chunk]
        s = test.loc[test.customer_ID.isin(cc)].shape[0]
        rows.append(s)
    if verbose != '': print( rows )
    return rows,chunk

# COMPUTE SIZE OF 4 PARTS FOR TEST DATA
NUM_PARTS = 4
TEST_PATH = '../data/final/test.parquet'

print(f'Reading test data...')
test = read_file(path = TEST_PATH, usecols = ['customer_ID'])
customers = test[['customer_ID']].drop_duplicates().sort_index().values.flatten()
rows,num_cust = get_rows(customers, test[['customer_ID']], NUM_PARTS = NUM_PARTS, verbose = 'test')

Reading test data...


NameError: name 'pd' is not defined

In [25]:
customers == customers[1]

False

In [1]:
len(customers), len(test)

NameError: name 'customers' is not defined

In [5]:
FEATURES = test.columns
FEATURES=FEATURES[1:]
VER =""
FOLDS= 5

In [6]:
#INFER TEST DATA IN PARTS
skip_rows = 0
skip_cust = 0
test_preds = []

for k in range(NUM_PARTS):
    
    # READ PART OF TEST DATA
    print(f'\nReading test data...')
    test = read_file(path = TEST_PATH)
    test = test.iloc[skip_rows:skip_rows+rows[k]]
    skip_rows += rows[k]
    print(f'=> Test part {k+1} has shape', test.shape )
    
    # PROCESS AND FEATURE ENGINEER PART OF TEST DATA
    if k==NUM_PARTS-1: test = test.loc[customers[skip_cust:]]
    else: test = test.loc[customers[skip_cust:skip_cust+num_cust]]
    skip_cust += num_cust
    
    #TEST DATA FOR XGB
    X_test = test[FEATURES]
    dtest = xgb.DMatrix(data=X_test)
    del X_test
    gc.collect()

    #INFER XGB MODELS ON TEST DATA
    model = xgb.Booster()
    model.load_model(f'../models/XGB_V_fold0.xgb')
    preds = model.predict(dtest)
    for f in range(1,FOLDS):
        model.load_model(f'XGB_V_fold{f}.xgb')
        preds += model.predict(dtest)
    preds /= FOLDS
    test_preds.append(preds)

    #CLEAN MEMORY
    del dtest, model
    _ = gc.collect()


Reading test data...
shape of data: (11363762, 103)
=> Test part 1 has shape (2841209, 103)


KeyError: "None of [Index(['00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7',\n       '00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5',\n       '0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8',\n       '00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694',\n       '00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557',\n       '00004ffe6e01e1b688170bbd108da8351bc4c316eacfef48643ee028ab947afc',\n       '00007cfcce97abfa0b4fa0647986157281d01d3ab90de919398cc0ba335710b5',\n       '000089cc2a30dad8e6ba39126f9d86df6088c9f975093a2e80b555b72b232406',\n       '00008f50a1dd76fa211ba36a2b0d5a1b201e4134a5fd53d04f1693d4abb70b2b',\n       '0000b48a4f27dc1d61e78d081678e811620300b88eb3ab25d3234e2a2a3c9200',\n       ...\n       '40018487b5dd7759be9ad2a0525c239fdb42b02f66f378b1a32122142a18aeff',\n       '400188efa1835034caab6e5c57821aae643091f703549deb299f9051fab7d4b2',\n       '40018a754f79a5168e231e4683f7ecd3c98faeca55c7eabf

In [ ]:
# WRITE SUBMISSION FILE
#test_preds = np.concatenate(test_preds)
test = pd.DataFrame(index=customers,data={'prediction':test_preds})
sub = pd.read_csv('../data/raw/sample_submission.csv')[['customer_ID']]
hex_to_int = lambda x: int(x, 16)
sub['customer_ID_hash'] = sub['customer_ID'].str[-16:].apply(hex_to_int).astype("int64")
sub = sub.set_index('customer_ID_hash')
sub = sub.merge(test[['prediction']], left_index=True, right_index=True, how='left')
sub = sub.reset_index(drop=True)

# DISPLAY PREDICTIONS
sub.to_csv(f'submission_xgb_v{VER}.csv',index=False)
print('Submission file shape is', sub.shape )
sub.head()

ValueError: Length of values (11363762) does not match length of index (924621)

In [ ]:
# PLOT PREDICTIONS
plt.hist(sub.to_pandas().prediction, bins=100)
plt.title('Test Predictions')
plt.show()